Given Twitter US Airline Sentiment Dataset, which contains data for over 14000 tweets.
Your task is to predict the sentiment of the tweet i.e. positive, negative or neutral.

### Imports Needed

In [1]:
# Data Manipulation
import numpy as np
import string

# stopwords and methods for preprocessing text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag 
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# sklearn classifiers and methods
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

### Read train and test data

In [31]:
def load_data():
    df_train = pd.read_csv('train.csv', encoding='utf-8')
    df_test = pd.read_csv('test.csv', encoding='utf-8')
    return df_train, df_test

In [32]:
# Call load_data
df_train ,df_test = load_data()

In [33]:
df_train.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [34]:
df_test.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [36]:
df_train.shape

(10980, 12)

In [48]:
df_test.shape

(3660, 11)

### Return list of all words of text

In [66]:
def get_words_list(tweet):
    words_list = list(tweet.split())
    return words_list

### Build training and testing documents

In [68]:
def build_train_and_test_documents():
    training_documents = []
    testing_documents = []

    # Build training _documents
    for tweet in df_train['text']:
        training_documents.append(get_words_list(tweet))

    # Build testing _documents
    for tweet in df_test['text']:
        testing_documents.append(get_words_list(tweet))
        
    return training_documents, testing_documents

In [69]:
# Calling build_train_and_test_documents
train_documents, test_documents = build_train_and_test_documents()

In [70]:
print(len(train_documents))
train_documents[0]

10980


['@SouthwestAir',
 'I',
 'am',
 'scheduled',
 'for',
 'the',
 'morning,',
 '2',
 'days',
 'after',
 'the',
 'fact,',
 'yes..not',
 'sure',
 'why',
 'my',
 'evening',
 'flight',
 'was',
 'the',
 'only',
 'one',
 'Cancelled',
 'Flightled']

In [71]:
print(len(test_documents))
test_documents[0]

3660


['@AmericanAir',
 'In',
 'car',
 'gng',
 'to',
 'DFW.',
 'Pulled',
 'over',
 '1hr',
 'ago',
 '-',
 'very',
 'icy',
 'roads.',
 'On-hold',
 'with',
 'AA',
 'since',
 '1hr.',
 "Can't",
 'reach',
 'arpt',
 'for',
 'AA2450.',
 'Wat',
 '2',
 'do?']

### Get sentiments corresponding to each tweet

In [72]:
sentiments = list(df_train['airline_sentiment'])

In [75]:
print(len(sentiments))
sentiments

10980


['negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive'

### StopWords and punctuations

In [78]:
stop = stopwords.words('english')
punctuation = string.punctuation
stop += punctuation
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

#### pos_tag gives parts of speech tag but in lemmatize we need different form of tag. So need to convert the tag given by pos_tag

In [79]:
def get_simpler_pos(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN # Assume rest everything is noun for simplicity

### Preprocessing
- Remove stopwords
- Remove punctuations and 
- Convert words into their root words i.e Lemmatization
- Convert into lowercase

In [80]:
def preprocess(words): # get list of words as documnets
    lemmatizer = WordNetLemmatizer()
    output_words = []
    for w in words:
        if w.lower() not in stop:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simpler_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [81]:
train_documents = [preprocess(doc) for doc in train_documents]

In [82]:
test_documents = [preprocess(doc) for doc in test_documents]

### Now we use CountVectorizer or TFidfVectorizer to transform data in format required by sklearn Classifier
- Since Vectorizer requires array of strings. So first we need to convert list of words into a single string.

In [84]:
train_documents = [" ".join(doc) for doc in train_documents]

In [85]:
test_documents = [" ".join(doc) for doc in test_documents]

In [86]:
train_documents[0]

'@southwestair schedule morning, 2 day fact, yes..not sure even flight one cancelled flightled'

In [123]:
def Vectorize(x_train, x_test, test_documents):
    '''
        We build vocab (Feature-set) using x_train only i.e call fit_transform on x_train only and then we transform 
        test data using values it learn.
    '''
    count_vec = TfidfVectorizer(max_features=3500) #, max_df=0.85)
    
    x_train_features = count_vec.fit_transform(x_train)
    
    # transform test data
    x_test_features = count_vec.transform(x_test)
    test_doc_features = count_vec.transform(test_documents)
    
    return x_train_features, x_test_features, test_doc_features

### train_test_split

In [124]:
x_train, x_test, y_train, y_test = train_test_split(train_documents, sentiments,random_state=0)

In [125]:
# Call Vectorize
x_train_features, x_test_features, test_doc_features = Vectorize(x_train, x_test, test_documents)

In [126]:
# Above 3 are sparse matrices
x_train_features

<8235x3500 sparse matrix of type '<class 'numpy.float64'>'
	with 77642 stored elements in Compressed Sparse Row format>

### Use sklearn classifier

In [127]:
svc = SVC()

In [128]:
svc.fit(x_train_features, y_train)

SVC()

In [129]:
svc.score(x_test_features, y_test)

0.770127504553734

### Predict results

In [130]:
y_pred = svc.predict(test_data).reshape(-1,1) # reshape into column

NameError: name 'test_data' is not defined

In [ ]:
np.savetxt('predictions.csv', y_predictions, delimiter=',', fmt='%s')